# To Read
El notebook de este TFM posee la siguiente estructura, detallada a continuación para su comprensión:
- Import Library. Importaciones de Librerías
- Load Data (XML) function. Contiene las funciones necesarias para la lectura del fichero fuente XML de Orphadata, y, a partir de ahí, poder trabajar con los datos.
- Exploratory Data Analysis (EDA) & Feature Engineering functions. Funciones comunes para los dos experimentos, asociadas a la Exploración de los datos, limpieza y preparación de los mismos.
- Test/Experiment: Recomendation functions. Funciones asociadas al Experimento, en el que se trabaja con un Sistema Recomendador. Contiene:  ...... bla bla bla bla bala
- Notebook Main: Common Tasks: Es el MAIN principal del notebook. Contiene las llamadas a las diferentes funciones definidas en el notebook para la obtención de los resultados. 



# Import Library
Se importan todas las librerías necesarias para trabajar en el presente Notebook para los dos experimentos que contiene

In [64]:
import xmltodict
import pandas as pd
import numpy as np
import sqlite3
import random
import sklearn
from sklearn.neighbors import NearestNeighbors

# Load Data (XML) function

load_from_xml: Función que extrae en un dataframe los datos de Enfermedades-Síntomas-Frecuencias del XML de Orphadata  (almacenado en una ruta concreta). Es llamada únicamente al principio para trabajar, pues luego se continua trabajando con dataframes y csvs

In [65]:
def load_from_xml (path):
    xml=open(path, encoding='ISO-8859-1')
    xmldict = xmltodict.parse(xml.read())
    df_enfermedades = pd.DataFrame()
    lista_enfer_sinto_prob=[]
    first_tree=xmldict["JDBOR"]["HPODisorderSetStatusList"]["HPODisorderSetStatus"]
    id=0
    for nodo in xmldict["JDBOR"]["HPODisorderSetStatusList"]["HPODisorderSetStatus"]:
     
        enfermedad=_clean_name(nodo["Disorder"]["Name"])
        sec_tree= nodo["Disorder"]["HPODisorderAssociationList"]
        if (len(sec_tree)==2):
            tam_sintomas=len(sec_tree["HPODisorderAssociation"])
            i=0
            registro_enfer_sinto_prob=[]
            while (i<tam_sintomas):
                registro_enfer_sinto_prob.append(id)
                registro_enfer_sinto_prob.append(enfermedad)
                registro_enfer_sinto_prob.append(sec_tree["HPODisorderAssociation"][i]["HPO"]["HPOTerm"])
                frecuencia=_clean_name(sec_tree["HPODisorderAssociation"][i]["HPOFrequency"]["Name"])
                registro_enfer_sinto_prob.append(frecuencia)
                lista_enfer_sinto_prob.append (registro_enfer_sinto_prob)
                registro_enfer_sinto_prob=[]
                i=i+1
        id=id+1
    df_enfermedades = pd.DataFrame(lista_enfer_sinto_prob)
    df_enfermedades = df_enfermedades.rename(columns={0:'Id_Enfermedad',1:'Enfermedad', 2:'Sintoma', 3:"Frecuencia"})
    
    return df_enfermedades        

_clean_name: Función privada, que usa load_from_xml, para eliminar carácteres innecesarios en los nombres encontrados en el XML fuente (para enfermedades y frecuencias) 

In [66]:
def _clean_name (cadena):
    
    cadena_str= str(cadena)
    cadena_str=cadena_str[26:]
    cadena_str = cadena_str.replace('}', '',1)
    cadena_str = cadena_str.replace('\'','')
    
    return cadena_str

# Exploratory Data Analysis (EDA) & Feature Engineering functions

eda_data: Función que dado un dataframe de entrada, aplica Exploratory Data Analysis. En concreto: elimina registros duplicados, los nulos, y los registros que contienen síntomas que sólo aparecen menos de 50 veces en nuestra muestra

In [67]:
def eda_data(data): 
    
    data=data.drop_duplicates()
    data=data.dropna()
    vc = data["Sintoma"].value_counts()
    vector=vc[vc < 50].index
    for a in vector:
        indexNames = data [ data["Sintoma"] == a ].index
        for b in indexNames:
            data.drop(b , inplace=True, axis=0)
  
    return data 

selection_data_by_frecuency: Función que dado un dataframe de entrada, se queda con los registros de Enfermedad-Sintoma-Frecuencia que tengan una frecuencia "Muy frecuente", "Frecuente", "Obligatorio" y "Ocasional". Elimina pues los registros con frecuencia Muy poco frecuente y Excluyente.

In [68]:
def selection_data_by_frecuency(data):
    
   
    data=data[(data['Frecuencia']=="Muy frecuente (99-80%)") |
              (data['Frecuencia']=="Frecuente (79-30%)") |
              (data['Frecuencia']=="Obligatorio (100%)") |
              (data['Frecuencia']=="Ocasional (29-5%)")
              
             ]
        
    return data


# Test/Experiment: Recomendation functions

generate_data_matrix_ratings: Función que dado un dataframe de entrada (con todas los registros existentes entre enfermedades, sus síntomas y la frecuencia de aparición), genera una matriz de enfermedades x sintomas, formada por valores únicamente en celdas donde un síntoma concreto (columna) aparezca en la enfermedad. El valor que tendrá dependerá de la frecuencia de aparición de dicho síntoma en la enfermedad (puntos) de tal forma que si es una frecuencia alta la aparición del síntoma en la enfermedad, tendrá más puntuación que una frecuencia más baja. 
Se persigue con esto disponer de una matriz de puntuaciones/ratings donde cruzamos todos los síntomas con todas las enfermedades, con un conjunto de puntuaciones.

In [69]:
def generate_data_matrix_ratings (data, repeticiones):
    
    sintomas=data.iloc[:,1]
    sintomas_sin_repe=sintomas.drop_duplicates()
    sintomas_sin_repe=sintomas_sin_repe.sort_values(ascending
                              = True)
    df_train=pd.DataFrame(columns=sintomas_sin_repe)
  
    data_agrupado = (data.groupby("Enfermedad")
         .agg({"Sintoma": np.array, "Frecuencia": np.array})
         .reset_index()
         )
    
    z=0
    j=0
    while (z<repeticiones):
    
  #  print ("entra")
        i=0
        for a in data_agrupado["Enfermedad"]:
           # print ("Enfermedad: ", a)

            lst = [0] * ((len(sintomas_sin_repe)))
           
            df_train.loc[len(df_train)] = lst
           

           
            pos=0
            for b in data_agrupado["Sintoma"][i]:
                #valor_aleatorio = random.random()
                frecuencia=data_agrupado["Frecuencia"][i][pos]
  
                if (frecuencia=="Muy frecuente (99-80%)"):
                        valor_entero=3
                elif (frecuencia=="Frecuente (79-30%)"):
                        valor_entero=2
                elif (frecuencia=="Obligatorio (100%)"):
                    valor_entero=4
                elif (frecuencia=="Ocasional (29-5%)"):
                    valor_entero=1
                    
                df_train[b][j]=valor_entero
               
                pos=pos+1
            j=j+1
            i=i+1
        z=z+1
        print ("VUELTA: ", z)
        print ("Tamaño: ",j)
    return df_train      

Matriz de recomendaciones



Predicción a partir de matriz de recomendaciones

Pearson

In [70]:
def predict_similitud_entre_usuarios_by_pearson (df_train_generated,sintoma,elementos):
    
    id_sintoma = df_Sintomas[df_Sintomas['Sintoma'] == sintoma].index.values[0]

    
    df_transpuesta=df_train_generated
    correlaciones = df_transpuesta.corr(method='pearson')

    MediasUsers = df_transpuesta.mean(numeric_only=True,skipna=True)
    DesviacionesUsers = df_transpuesta.std(numeric_only=True,skipna=True)
    df_ValoracionesNorm = df_transpuesta[MediasUsers.index].sub(MediasUsers, axis='columns')

    df_ValoracionesNorm = df_ValoracionesNorm.div(DesviacionesUsers, axis='columns')

    Numerador = df_ValoracionesNorm.dot(correlaciones)
    Numerador.sort_values(0,ascending=False)
    Denominador = correlaciones.abs().sum()
    Cociente = Numerador.div(Denominador, axis='columns')

    Valoraciones = Cociente.mul(DesviacionesUsers,axis='columns').add(MediasUsers, axis='columns')

    a=Valoraciones[id_sintoma].sort_values(ascending=False).head(elementos)
    
    
    
    
    
    vector=pd.DataFrame(a)
    vector=vector.reset_index()
    lista=_aparece_y_como (vector,id_sintoma, df_Enfermedades)
    
    
 
    
    return lista,a

Knearest

In [71]:
def _aparece_y_como (scoring_enfermedades,id_Sintoma, df_Enfermedades):
    

    
    j=0
    enfermedades=[]
    while (j<len(scoring_enfermedades)):
        enfermedad=[]
        id_enfermedad=scoring_enfermedades["index"][j]
        scoring=scoring_enfermedades[id_Sintoma][j]
        
        enfermedad.append(id_enfermedad)
       
        enfermedad.append(df_Enfermedades[df_Enfermedades["index"]==id_enfermedad]["Enfermedad"].values[0])
   
        enfermedad.append(scoring)
        lista=df_EnfeySinto_select[df_EnfeySinto_select["Enfermedad"]==
                                   df_Enfermedades.loc[id_enfermedad][1]]
        lista=lista.reset_index()
        sintoma= df_Sintomas.loc[id_Sintoma].Sintoma
        
        i=0
        while i<len(lista):
         
            if lista["Sintoma"][i]==sintoma:
                enfermedad.append(lista["Frecuencia"][i])
            
            i=i+1  
        j=j+1
        enfermedades.append(enfermedad)
    return enfermedades

Con más sintomas el pearson

In [72]:


#common = set(first).intersection(second)
##HACE INTERSECCIÓN DE VARIOS SINTOMAS
def predict_similitud_entre_usuarios_by_pearson_multi (sintomas,df_transpuesta):
    comunes=[]
    primera_iter=True


    for i in sintomas:
        vector=[]
        lista,vector=predict_similitud_entre_usuarios_by_pearson(df_transpuesta,i,200)
        #listado_completo,users_predictions,ratings_train, ratings_test, sim_matrix, v=predict_collaborative_filtering_ser_based(
    #df_matrix.values, i,10)
        v=vector.reset_index()
        
        sintoma=v["index"]
        sintoma=list(sintoma)
      
        if primera_iter:
            comunes=sintoma
            primera_iter=False

        else:            
            comunes = set(comunes).intersection(sintoma)
        
    return comunes

Con más síntomas el el user based

In [73]:


#common = set(first).intersection(second)
##HACE INTERSECCIÓN DE VARIOS SINTOMAS
def collaborative_filtering_user_based_multi (sintomas,df_transpuesta):
    comunes=[]
    primera_iter=True
    df_matrix=df_transpuesta.transpose()
    
    #init_collaborative_filtering_user_based (df_matrix.values)
    
    for i in sintomas:
        vector=[]
        #lista,vector=predict_similitud_entre_usuarios_by_pearson(df_transpuesta,i,20)
        listado_completo,users_predictions, v=predict_collaborative_filtering_user_based(i,200)
        #v=vector.reset_index()
        
        sintoma=v["index"]
        sintoma=list(sintoma)
        #print(sintoma)
      
        if primera_iter:
            comunes=sintoma
            primera_iter=False

        else:            
            comunes = set(comunes).intersection(sintoma)
            
    return comunes


_saca_enfermedades: Función que devuelve un vector de Enfermedades (su literal), dado un vector de ids. Es utilizado únicamente par ver resultados y comparar, ya que es más fácil ver los resultados de los sistemas recomendador con el propio literal de la Enfermedad que con su Id.

In [74]:
def _saca_enfermedades (vector,df_Enfermedades):
    
    enfermedades=[]
    for j in vector:
        enfermedades.append(df_Enfermedades[df_Enfermedades["index"]==j]["Enfermedad"])
        
    return enfermedades

_rename_col: Función que, dado un dataframe con un conjunto de columnas, modifica el nombre de dichas columnas para poder trabajar con su Id, en lugar de literales.




In [75]:
def _rename_col(df):
    
    df.rename(columns={'Sintoma':'Enfermedades'},
               inplace=True)
    columnas=len(df.columns)
    i=0
    while (i<columnas):
        df = df.rename(columns={df.columns[i]:i})
        i=i+1
        
        
    return df

# Notebook Main

## Common Tasks



Vayamos por pasos:

Se comienza con la lectura del fichero XML fuente de Orphadata, llamando a la función definida anteriormente

In [81]:
df_EnfeySinto=load_from_xml("../data/01_raw/enfermedades.xml")


A modo visual, vemos qué hemos obtenido de esta lectura, tanto a nivel de Enfermedades, como síntomas y frecuencias de aparición

In [82]:
print ("Datos de Lectura del XML Fuente Orphadata")
print ("Enfermedades: ", df_EnfeySinto["Enfermedad"].nunique())
print ("Sintomas: ", df_EnfeySinto["Sintoma"].nunique())
print ("Frecuencias: ", df_EnfeySinto["Frecuencia"].nunique())  

Datos de Lectura del XML Fuente Orphadata
Enfermedades:  4262
Sintomas:  8303
Frecuencias:  6


In [83]:
df_EnfeySinto

,Id_Enfermedad,Enfermedad,Sintoma,Frecuencia
0,0,Enfermedad de Alexander,Macrocephaly,Muy frecuente (99-80%)
1,0,Enfermedad de Alexander,Intellectual disability,Muy frecuente (99-80%)
2,0,Enfermedad de Alexander,Seizure,Muy frecuente (99-80%)
3,0,Enfermedad de Alexander,Spasticity,Muy frecuente (99-80%)
4,0,Enfermedad de Alexander,Agenesis of corpus callosum,Muy frecuente (99-80%)
...,...,...,...,...
112684,4263,Nevo de cabello lanoso,Brachydactyly,Ocasional (29-5%)
112685,4263,Nevo de cabello lanoso,Widely-spaced incisors,Ocasional (29-5%)
112686,4263,Nevo de cabello lanoso,Persistent pupillary membrane,Ocasional (29-5%)
112687,4263,Nevo de cabello lanoso,Enlarged vestibular aqueduct,Ocasional (29-5%)


In [84]:
%%time
df_EnfeySinto_cleaned=eda_data(df_EnfeySinto)


CPU times: user 4min 49s, sys: 568 ms, total: 4min 49s
Wall time: 4min 49s


In [85]:
print ("Después de EDA")
print ("Enfermedades: ", df_EnfeySinto_cleaned["Enfermedad"].nunique())
print ("Sintomas: ", df_EnfeySinto_cleaned["Sintoma"].nunique())
print ("Frecuencias: ", df_EnfeySinto_cleaned["Frecuencia"].nunique())  

Después de EDA
Enfermedades:  4187
Sintomas:  483
Frecuencias:  6


In [86]:
df_EnfeySinto_cleaned


,Id_Enfermedad,Enfermedad,Sintoma,Frecuencia
0,0,Enfermedad de Alexander,Macrocephaly,Muy frecuente (99-80%)
1,0,Enfermedad de Alexander,Intellectual disability,Muy frecuente (99-80%)
2,0,Enfermedad de Alexander,Seizure,Muy frecuente (99-80%)
3,0,Enfermedad de Alexander,Spasticity,Muy frecuente (99-80%)
4,0,Enfermedad de Alexander,Agenesis of corpus callosum,Muy frecuente (99-80%)
...,...,...,...,...
112675,4262,Artrogriposis múltiple congénita miogénica aut...,Microcephaly,Excluyente (0%)
112676,4262,Artrogriposis múltiple congénita miogénica aut...,Polyhydramnios,Excluyente (0%)
112679,4263,Nevo de cabello lanoso,Fine hair,Muy frecuente (99-80%)
112684,4263,Nevo de cabello lanoso,Brachydactyly,Ocasional (29-5%)


In [90]:
df_EnfeySinto_select=selection_data_by_frecuency(df_EnfeySinto_cleaned)


In [91]:
df_EnfeySinto_select

,Id_Enfermedad,Enfermedad,Sintoma,Frecuencia
0,0,Enfermedad de Alexander,Macrocephaly,Muy frecuente (99-80%)
1,0,Enfermedad de Alexander,Intellectual disability,Muy frecuente (99-80%)
2,0,Enfermedad de Alexander,Seizure,Muy frecuente (99-80%)
3,0,Enfermedad de Alexander,Spasticity,Muy frecuente (99-80%)
4,0,Enfermedad de Alexander,Agenesis of corpus callosum,Muy frecuente (99-80%)
...,...,...,...,...
112669,4262,Artrogriposis múltiple congénita miogénica aut...,Depressed nasal bridge,Ocasional (29-5%)
112671,4262,Artrogriposis múltiple congénita miogénica aut...,Postnatal growth retardation,Ocasional (29-5%)
112672,4262,Artrogriposis múltiple congénita miogénica aut...,Gastrostomy tube feeding in infancy,Ocasional (29-5%)
112679,4263,Nevo de cabello lanoso,Fine hair,Muy frecuente (99-80%)


In [92]:
  print ("Después de Selection")
  print ("Enfermedades: ", df_EnfeySinto_select["Enfermedad"].nunique())
  print ("Sintomas: ", df_EnfeySinto_select["Sintoma"].nunique())
  print ("Frecuencias: ", df_EnfeySinto_select["Frecuencia"].nunique())  


Después de Selection
Enfermedades:  4173
Sintomas:  483
Frecuencias:  4


Ya disponemos de la información preparada para avanzar con los experimentos. Se comienza primero con el experimento que se planteo de forma inicial

## Recomendator

### Datos Ratings Matrix

In [93]:
df_EnfeySinto_select_no_id=df_EnfeySinto_select.drop("Id_Enfermedad", axis=1)
df_EnfeySinto_select_no_id


,Enfermedad,Sintoma,Frecuencia
0,Enfermedad de Alexander,Macrocephaly,Muy frecuente (99-80%)
1,Enfermedad de Alexander,Intellectual disability,Muy frecuente (99-80%)
2,Enfermedad de Alexander,Seizure,Muy frecuente (99-80%)
3,Enfermedad de Alexander,Spasticity,Muy frecuente (99-80%)
4,Enfermedad de Alexander,Agenesis of corpus callosum,Muy frecuente (99-80%)
...,...,...,...
112669,Artrogriposis múltiple congénita miogénica aut...,Depressed nasal bridge,Ocasional (29-5%)
112671,Artrogriposis múltiple congénita miogénica aut...,Postnatal growth retardation,Ocasional (29-5%)
112672,Artrogriposis múltiple congénita miogénica aut...,Gastrostomy tube feeding in infancy,Ocasional (29-5%)
112679,Nevo de cabello lanoso,Fine hair,Muy frecuente (99-80%)


In [94]:
df_Enfermedades=df_EnfeySinto_select_no_id.groupby (["Enfermedad"]).count().reset_index()
df_Enfermedades=df_Enfermedades.drop(["Sintoma","Frecuencia"], axis=1)
df_Enfermedades=df_Enfermedades.reset_index()
df_Enfermedades

,index,Enfermedad
0,0,AICA ribosiduria
1,1,ALG1-CDG
2,2,ALG11-CDG
3,3,ALG12-CDG
4,4,ALG13-CDG
...,...,...
4168,4168,Xeroderma pigmentoso
4169,4169,Xeroderma pigmentoso tipo variante
4170,4170,Zigomicosis
4171,4171,síndrome de talla baja-sordera-disfunción neut...


In [95]:
df_Sintomas=df_EnfeySinto_select_no_id.groupby (["Sintoma"]).count().reset_index()
df_Sintomas=df_Sintomas.drop(["Enfermedad","Frecuencia"], axis=1)
df_Sintomas

,Sintoma
0,Abdominal distention
1,Abdominal pain
2,Abnormal bleeding
3,Abnormal blistering of the skin
4,Abnormal cardiac septum morphology
...,...
478,Weight loss
479,Wide intermamillary distance
480,Wide mouth
481,Wide nasal bridge


In [96]:
%%time
df_train_generated=generate_data_matrix_ratings (df_EnfeySinto_select_no_id, 1)



VUELTA:  1
Tamaño:  4173
CPU times: user 5.87 s, sys: 33.3 ms, total: 5.9 s
Wall time: 5.9 s


In [97]:

df_train_generated


Sintoma,Abdominal distention,Abdominal pain,Abnormal bleeding,Abnormal blistering of the skin,Abnormal cardiac septum morphology,Abnormal cerebral white matter morphology,Abnormal dermatoglyphics,Abnormal enzyme/coenzyme activity,Abnormal facial shape,Abnormal fingernail morphology,...,Visual impairment,Visual loss,Vomiting,Waddling gait,Webbed neck,Weight loss,Wide intermamillary distance,Wide mouth,Wide nasal bridge,Wide nose
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,0
1,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,4,1,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4168,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4169,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4170,0,1,0,1,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
4171,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [98]:
df_train_generated=_rename_col(df_train_generated)
df_train_generated



Sintoma,0,1,2,3,4,5,6,7,8,9,...,473,474,475,476,477,478,479,480,481,482
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,0
1,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,4,1,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4168,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4169,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4170,0,1,0,1,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
4171,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [99]:
df_train_generated_T=df_train_generated.transpose()


In [100]:
df_train_generated_T

,0,1,2,3,4,5,6,7,8,9,...,4163,4164,4165,4166,4167,4168,4169,4170,4171,4172
Sintoma,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
479,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
480,3,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


## Collaborative Filtering: User based

In [122]:
def init_collaborative_filtering_user_based (ratings_matrix):
    #Recibimos la matriz "ratings_matrix" que está compuesta por el cruce entre todos los síntomas y todos las
    #enfermedades; y que tiene los valores 0 (cuando no hay relación) y 1,2,3 y 4 en función de la frecuencia
    #de releación entre el síntoma y la enfermedad. Por ejemplo, si la frecuencia es "muy frecuente", vendrá
    #con un 4 en la celda de relación entre el síntoma y la enfermedad.
    
    
    
    #Ahora calculamos la matriz la similitud entre enfermedades, utilizando la función cosine_similarity (distancia
    #del coseno (vectores))
   
    sim_matrix= sklearn.metrics.pairwise.cosine_similarity(ratings_matrix)
    #Ya tenemos la matriz de similitudes. 
    
    #Ahora ya podemos calcular las recomendaciones. Podemos predecir las enfermedades recomendadas a partir 
    #de una matriz calculada en donde hay relación entre cada enfermedad y síntoma (con un scoring)
    #Es la matriz "sintomas_k"
    #Por eso usamos "ratings_matrix" que tiene las "puntuaciones" entre síntoma y enfermedad
    #Según apuntes... 
    #Cada rating se multiplica por el factor de similitud de enfermedad que dio el rating. 
    #La predicción final por enfermedad será igual a la suma del peso de los ratings dividido por la “suma ponderada”.
    sintomas_k = sim_matrix.dot(ratings_matrix) / np.array([np.abs(sim_matrix).sum(axis=1)]).T
    # Producto de Matriz de similitud (sim_matrix) con la matrix inicial (ratings_matrix)
    #/ (sumatoria de cada fila de ratings) con Transpuesta
    
    print ("matriz de similitudes entre síntomas. Por eso es 483x483")
    print (sim_matrix)
    print (sim_matrix.shape)
    print ("matriz de recomendaciones entre síntoma y enfermedades. Por eso es 483x4173")
    print (sintomas_k)
    print (sintomas_k.shape)
    #Preparamos BBDD
    conn = sqlite3.connect('test_database')
    c = conn.cursor()
    #Hacemos transpose para guardar o dará error en "demasiadas columnas"
    sintomas_k=sintomas_k.transpose()
    df = pd.DataFrame(sintomas_k)
   # df=df.transpose()
    df.to_sql('scorings_tfm_T', conn, if_exists='replace', index = False)
    
    df2 = pd.DataFrame(sim_matrix)
    #df2=df2.transpose()
    df2.to_sql('sim_matrix_tfm_T', conn, if_exists='replace', index = False)
   


    

SOLO UN SINTOMA

In [123]:
init_collaborative_filtering_user_based (df_train_generated_T.values)



matriz de similitudes entre síntomas. Por eso es 483x483
[[1.         0.24714085 0.00896437 ... 0.         0.         0.02341284]
 [0.24714085 1.         0.09707473 ... 0.01394178 0.02037137 0.00559271]
 [0.00896437 0.09707473 1.         ... 0.         0.00317958 0.        ]
 ...
 [0.         0.01394178 0.         ... 1.         0.18243109 0.14587192]
 [0.         0.02037137 0.00317958 ... 0.18243109 1.         0.11677951]
 [0.02341284 0.00559271 0.         ... 0.14587192 0.11677951 1.        ]]
(483, 483)
matriz de recomendaciones entre síntoma y enfermedades. Por eso es 483x4173
[[0.0408232  0.07877217 0.06944562 ... 0.12429085 0.02014754 0.        ]
 [0.01589985 0.06918101 0.04982591 ... 0.23836104 0.0128197  0.        ]
 [0.02007494 0.09350656 0.06238897 ... 0.13350724 0.01519163 0.00371038]
 ...
 [0.22874629 0.14428422 0.16739733 ... 0.00391929 0.02247898 0.00376148]
 [0.10006087 0.14691641 0.14493963 ... 0.00456519 0.0337019  0.00631822]
 [0.09869133 0.13961413 0.1541426  ... 0.0

In [124]:
def predict_collaborative_filtering_user_based(sintoma, elementos):
 
     #Nos viene el Síntoma en la variable "sintoma" (ejemplo "Cough") y el número de elementos enfermedades
    #que queremos que nos recomiende con ese síntoma. Vamos a pensar que son 10 para el ejemplo.
    
    #Sacamos el Id del Sintoma (al tener el texto)
    id_sintoma = df_Sintomas[df_Sintomas['Sintoma'] == sintoma].index.values[0]
    
    
    
    conn = sqlite3.connect('test_database')
    c = conn.cursor()
    c.execute('''  SELECT * FROM scorings_tfm_T
          ''')
    #Predicciones (las recomendaciones!)
    #Obtenemos c.fetchall la matriz y montamos dataframe
    sintomas_k = pd.DataFrame(c.fetchall())  
    print ("matriz de recomendaciones entre síntoma y enfermedades. Por eso es 4173x483")
   
    print (sintomas_k)
    print (sintomas_k.shape)
    
    c.execute('''  SELECT * FROM sim_matrix_tfm_T
          ''')
    #Predicciones (las recomendaciones!)
    #Obtenemos c.fetchall la matriz y montamos dataframe
    sim_matrix_tfm_T = pd.DataFrame(c.fetchall())  
    print ("matriz de similitudes entre enfermedades. Por eso es 483x483")

    print (sim_matrix_tfm_T)
    print (sim_matrix_tfm_T.shape)
    #transpose porque hemos tenido que hacer transpose para guardar
    sintomas_k=sintomas_k.transpose()
    #ahora la convertimos en array numpy para operar (hacer argsort)
    sintomas_k=sintomas_k.to_numpy()
       
    
    
    #Predicciones (las recomendaciones!)
    
    #Con la matrix de recomendaciones (síntomas vs enfermedades) cogemos únicamente el vector de enfermedades
    #relacionadas con el síntoma "id_sintoma"
    #Pero ojo,    hemos cogido un vector de enfermedades ordenadas!!! 
    sintoma_scoring=sintomas_k.argsort()[id_sintoma]
    #
    vector_id_enfermedad_scoring=[]
    #Inicializamos nuestro vector TOTAL que tendrá las parejas de "enfermedad" /scoring de recomendación
    #(valor en la matriz recomendaciones)
    for i, enfermedad_id in enumerate(sintoma_scoring[-elementos:]):  #Nos cogemos las 10 primeras enfermedades   
        #
        v=[]
    #vamos cogiendo cada pareja de id enfermedad y el scoring de recomendación  (la pareja la montamos en un vector=)
        v.append (enfermedad_id)
        v.append (sintomas_k[id_sintoma][enfermedad_id])
        print ("ranking de peor a mejor")
        print ("posición: ",i)
        print (v)
       
        vector_id_enfermedad_scoring.append(v)  #metemos la pareja en el vector TOTAL
      
    vector_id_enfermedad_scoring=pd.DataFrame(vector_id_enfermedad_scoring)
    #pasamos a data frame y luego montamos con la función interna "aparece y como" una visualización mejor.
    vector_id_enfermedad_scoring = vector_id_enfermedad_scoring.rename(columns={1:id_sintoma, 0:"index"})
    
    listado_completo=_aparece_y_como (vector_id_enfermedad_scoring,id_sintoma, df_Enfermedades)
  

    return listado_completo,sintomas_k, vector_id_enfermedad_scoring

In [125]:
listado_completo,users_predictions,v=predict_collaborative_filtering_user_based("Cough",10)


matriz de recomendaciones entre síntoma y enfermedades. Por eso es 4173x483
           0         1         2         3         4         5         6    \
0     0.040823  0.015900  0.020075  0.006777  0.059570  0.069889  0.083075   
1     0.078772  0.069181  0.093507  0.061627  0.112774  0.221177  0.121386   
2     0.069446  0.049826  0.062389  0.051579  0.108164  0.274688  0.137868   
3     0.101930  0.067367  0.124443  0.082578  0.141394  0.195223  0.139908   
4     0.020471  0.005900  0.017128  0.007142  0.026682  0.036400  0.032039   
...        ...       ...       ...       ...       ...       ...       ...   
4168  0.120956  0.248005  0.178637  0.345732  0.140150  0.171605  0.184312   
4169  0.009205  0.026291  0.049993  0.151200  0.009394  0.010939  0.007833   
4170  0.124291  0.238361  0.133507  0.205827  0.012902  0.033882  0.005148   
4171  0.020148  0.012820  0.015192  0.024546  0.026800  0.011853  0.031367   
4172  0.000000  0.000000  0.003710  0.000000  0.003943  0.006340  

In [126]:
#una de las variables es listado completo, que me pinta muy "bonito" todo. 
listado_completo


[[2207, 'Polimiositis', 0.6367147952553699, 'Muy frecuente (99-80%)'],
 [1423, 'Gripe aviar', 0.6382757844473661, 'Muy frecuente (99-80%)'],
 [2480,
  'Síndrome antisintetasa',
  0.6524342127715518,
  'Muy frecuente (99-80%)'],
 [1689,
  'Linfangioleiomiomatosis',
  0.6527724664337454,
  'Muy frecuente (99-80%)'],
 [4029, 'Tifus de las malezas', 0.6612850581965117, 'Muy frecuente (99-80%)'],
 [1207,
  'Enfermedad por anticuerpos anti-membrana basal glomerular',
  0.6746039663251943,
  'Muy frecuente (99-80%)'],
 [1189,
  'Enfermedad de los legionarios',
  0.7470164299505365,
  'Muy frecuente (99-80%)'],
 [1972,
  'Neumonía necrotizante estafilocócica',
  0.7474481267629827,
  'Muy frecuente (99-80%)'],
 [2715,
  'Síndrome de Hughes-Stovin',
  0.772552060870626,
  'Muy frecuente (99-80%)'],
 [1421,
  'Granulomatosis con poliangeítis',
  0.8535007869582253,
  'Frecuente (79-30%)']]

In [127]:
df_EnfeySinto_select[(df_EnfeySinto_select["Enfermedad"]=="Granulomatosis con poliangeítis") &
                     ((df_EnfeySinto_select["Sintoma"]=="Cough") )]

,Id_Enfermedad,Enfermedad,Sintoma,Frecuencia
8239,267,Granulomatosis con poliangeítis,Cough,Frecuente (79-30%)


 CON MULTI-SINTOMA

In [128]:
SINTOMAS_EJEMPLO = ['Cough','Dysphagia']

In [129]:
sintomas_comunes1=collaborative_filtering_user_based_multi (SINTOMAS_EJEMPLO,df_train_generated)
sintomas_comunes1

matriz de recomendaciones entre síntoma y enfermedades. Por eso es 4173x483
           0         1         2         3         4         5         6    \
0     0.040823  0.015900  0.020075  0.006777  0.059570  0.069889  0.083075   
1     0.078772  0.069181  0.093507  0.061627  0.112774  0.221177  0.121386   
2     0.069446  0.049826  0.062389  0.051579  0.108164  0.274688  0.137868   
3     0.101930  0.067367  0.124443  0.082578  0.141394  0.195223  0.139908   
4     0.020471  0.005900  0.017128  0.007142  0.026682  0.036400  0.032039   
...        ...       ...       ...       ...       ...       ...       ...   
4168  0.120956  0.248005  0.178637  0.345732  0.140150  0.171605  0.184312   
4169  0.009205  0.026291  0.049993  0.151200  0.009394  0.010939  0.007833   
4170  0.124291  0.238361  0.133507  0.205827  0.012902  0.033882  0.005148   
4171  0.020148  0.012820  0.015192  0.024546  0.026800  0.011853  0.031367   
4172  0.000000  0.000000  0.003710  0.000000  0.003943  0.006340  

matriz de recomendaciones entre síntoma y enfermedades. Por eso es 4173x483
           0         1         2         3         4         5         6    \
0     0.040823  0.015900  0.020075  0.006777  0.059570  0.069889  0.083075   
1     0.078772  0.069181  0.093507  0.061627  0.112774  0.221177  0.121386   
2     0.069446  0.049826  0.062389  0.051579  0.108164  0.274688  0.137868   
3     0.101930  0.067367  0.124443  0.082578  0.141394  0.195223  0.139908   
4     0.020471  0.005900  0.017128  0.007142  0.026682  0.036400  0.032039   
...        ...       ...       ...       ...       ...       ...       ...   
4168  0.120956  0.248005  0.178637  0.345732  0.140150  0.171605  0.184312   
4169  0.009205  0.026291  0.049993  0.151200  0.009394  0.010939  0.007833   
4170  0.124291  0.238361  0.133507  0.205827  0.012902  0.033882  0.005148   
4171  0.020148  0.012820  0.015192  0.024546  0.026800  0.011853  0.031367   
4172  0.000000  0.000000  0.003710  0.000000  0.003943  0.006340  

{399,
 493,
 645,
 1088,
 1090,
 1155,
 1156,
 1306,
 1369,
 1422,
 1956,
 2529,
 2786,
 2940,
 2997,
 3935}

In [131]:
print(_saca_enfermedades (sintomas_comunes1,df_Enfermedades))

[1088    Enfermedad de Gaucher
Name: Enfermedad, dtype: object, 2529    Síndrome de Alport ligado al cromosoma X-leiom...
Name: Enfermedad, dtype: object, 2786    Síndrome de Lynch
Name: Enfermedad, dtype: object, 1155    Enfermedad de Whipple
Name: Enfermedad, dtype: object, 1156    Enfermedad de Wilson
Name: Enfermedad, dtype: object, 645    Dermatomiositis juvenil
Name: Enfermedad, dtype: object, 1956    Necrólisis epidérmica tóxica
Name: Enfermedad, dtype: object, 1090    Enfermedad de Gaucher tipo 2
Name: Enfermedad, dtype: object, 493    Cáncer colorrectal familiar tipo X
Name: Enfermedad, dtype: object, 1422    Granulomatosis eosinofílica con poliangeítis
Name: Enfermedad, dtype: object, 399    Carcinoma de esófago
Name: Enfermedad, dtype: object, 2997    Síndrome de Williams
Name: Enfermedad, dtype: object, 1369    Fiebre hemorrágica del virus Ébola
Name: Enfermedad, dtype: object, 1306    Esclerosis sistémica cutánea difusa
Name: Enfermedad, dtype: object, 2940    Síndrome de 

In [137]:
df_EnfeySinto_select[(df_EnfeySinto_select["Enfermedad"]=="Síndrome oculocerebrorrenal de Lowe") &
                     ((df_EnfeySinto_select["Sintoma"]=="Cough") | (df_EnfeySinto_select["Sintoma"]=="Dysphagia")) ]

,Id_Enfermedad,Enfermedad,Sintoma,Frecuencia


## Similitud entre Usuarios: Correlación de Pearson

SOLO UN SINTOMA

In [138]:
listado_completo, vector_enfermedades_scoring=predict_similitud_entre_usuarios_by_pearson(
    df_train_generated,"Cough", 10)


In [139]:
listado_completo

[[2715,
  'Síndrome de Hughes-Stovin',
  0.6900752891765587,
  'Muy frecuente (99-80%)'],
 [1972,
  'Neumonía necrotizante estafilocócica',
  0.6590546575580434,
  'Muy frecuente (99-80%)'],
 [1421,
  'Granulomatosis con poliangeítis',
  0.6467186907484271,
  'Frecuente (79-30%)'],
 [1207,
  'Enfermedad por anticuerpos anti-membrana basal glomerular',
  0.6285446552044739,
  'Muy frecuente (99-80%)'],
 [345, 'Blastoma pulmonar', 0.6040833380603052, 'Muy frecuente (99-80%)'],
 [1189,
  'Enfermedad de los legionarios',
  0.5865656810364698,
  'Muy frecuente (99-80%)'],
 [1689,
  'Linfangioleiomiomatosis',
  0.5565176790671993,
  'Muy frecuente (99-80%)'],
 [215, 'Aspergilosis', 0.5408371321960675, 'Muy frecuente (99-80%)'],
 [4155,
  'Vasculitis urticarial hipocomplementémica',
  0.5121208314395281,
  'Frecuente (79-30%)'],
 [2480,
  'Síndrome antisintetasa',
  0.5036944902150464,
  'Muy frecuente (99-80%)']]

In [61]:
vector_enfermedades_scoring

2715    0.690075
1972    0.659055
1421    0.646719
1207    0.628545
345     0.604083
1189    0.586566
1689    0.556518
215     0.540837
4155    0.512121
2480    0.503694
Name: 115, dtype: float64

In [62]:
df_EnfeySinto_select[(df_EnfeySinto_select["Enfermedad"]=="Brucelosis")]

,Id_Enfermedad,Enfermedad,Sintoma,Frecuencia
94850,3797,Brucelosis,Hyperhidrosis,Frecuente (79-30%)
94851,3797,Brucelosis,Arthritis,Frecuente (79-30%)
94852,3797,Brucelosis,Splenomegaly,Frecuente (79-30%)
94853,3797,Brucelosis,Weight loss,Frecuente (79-30%)
94855,3797,Brucelosis,Anemia,Frecuente (79-30%)
94856,3797,Brucelosis,Fever,Frecuente (79-30%)
94857,3797,Brucelosis,Nausea,Frecuente (79-30%)
94858,3797,Brucelosis,Anorexia,Frecuente (79-30%)
94859,3797,Brucelosis,Hepatomegaly,Frecuente (79-30%)
94860,3797,Brucelosis,Arthralgia,Frecuente (79-30%)


 CON MULTI-SINTOMA

In [65]:
sintomas_comunes2=predict_similitud_entre_usuarios_by_pearson_multi (SINTOMAS_EJEMPLO,df_train_generated)
sintomas_comunes2

{11, 394, 399, 1090, 1306, 1360, 1369, 1422, 1956, 2005, 2211, 2529, 2940}

In [66]:
print(_saca_enfermedades (sintomas_comunes2,df_Enfermedades))

[2529    Síndrome de Alport ligado al cromosoma X-leiom...
Name: Enfermedad, dtype: object, 1090    Enfermedad de Gaucher tipo 2
Name: Enfermedad, dtype: object, 2211    Poliomielitis
Name: Enfermedad, dtype: object, 1956    Necrólisis epidérmica tóxica
Name: Enfermedad, dtype: object, 394    Carcinoma anaplásico de tiroides
Name: Enfermedad, dtype: object, 11    Acalasia idiopática
Name: Enfermedad, dtype: object, 1422    Granulomatosis eosinofílica con poliangeítis
Name: Enfermedad, dtype: object, 399    Carcinoma de esófago
Name: Enfermedad, dtype: object, 1360    Fiebre de Lassa
Name: Enfermedad, dtype: object, 2005    Neuropatía sensitivo-motora hereditaria, tipo ...
Name: Enfermedad, dtype: object, 1369    Fiebre hemorrágica del virus Ébola
Name: Enfermedad, dtype: object, 1306    Esclerosis sistémica cutánea difusa
Name: Enfermedad, dtype: object, 2940    Síndrome de Stevens-Johnson
Name: Enfermedad, dtype: object]


In [41]:
df_EnfeySinto_select[(df_EnfeySinto_select["Enfermedad"]=="Granulomatosis con poliangeítis") &
                     ((df_EnfeySinto_select["Sintoma"]=="Cough") )]

,Id_Enfermedad,Enfermedad,Sintoma,Frecuencia
8239,267,Granulomatosis con poliangeítis,Cough,Frecuente (79-30%)
